In [2]:
import torch
import re
import pandas as pd
import numpy as np

from transformers import AutoTokenizer, AutoModel, pipeline
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
from rank_bm25 import BM25Okapi
from langchain.text_splitter import CharacterTextSplitter
from llama_index import SimpleDirectoryReader
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM, pipeline
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\vmpletsos\Anaconda3\envs\guide\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# remove with regex the brackets and its content from a phrase
def remove_references(text):
    # strip sentenece
    text = text.lower().strip()
    # remove strange characters from documents
    text = text.replace('\xad', '')
    # remove dashes
    text = text.replace('-', '')
    # remove new lines
    text = text.replace('\n', '')
    # remove strange combinations
    text = text.replace('=-', '')
    # remove brackets
    text = re.sub(r'\(\d+\)', '', text)
    # remove figures
    text = re.sub(r'\(\w+ \d+\)', '', text)
    # remove references
    return re.sub(r'\[[\d\- ,]+\]', '', text)

In [4]:
documents = SimpleDirectoryReader('./Knowledge Base/').load_data();

In [5]:
corpus = [document.text for document in documents]

In [6]:
text_splitter = CharacterTextSplitter(        
    separator = ".",
    chunk_size = 300,
    chunk_overlap  = 150,
    length_function = len,
)

In [7]:
texts = text_splitter.create_documents(corpus);
texts = [remove_references(sentence.page_content) for sentence in texts]

Created a chunk of size 312, which is longer than the specified 300
Created a chunk of size 435, which is longer than the specified 300
Created a chunk of size 353, which is longer than the specified 300
Created a chunk of size 385, which is longer than the specified 300
Created a chunk of size 316, which is longer than the specified 300
Created a chunk of size 330, which is longer than the specified 300
Created a chunk of size 314, which is longer than the specified 300
Created a chunk of size 347, which is longer than the specified 300
Created a chunk of size 355, which is longer than the specified 300
Created a chunk of size 329, which is longer than the specified 300
Created a chunk of size 320, which is longer than the specified 300
Created a chunk of size 302, which is longer than the specified 300
Created a chunk of size 378, which is longer than the specified 300
Created a chunk of size 340, which is longer than the specified 300
Created a chunk of size 301, which is longer tha

In [8]:
df = pd.DataFrame({'passages': texts})
df.to_csv('new_passages.csv', index=False)

In [9]:
df = pd.read_csv('new_passages.csv')

In [10]:
# Load model from HuggingFace Hub
similarity_tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
similarity_model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

In [11]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [12]:
# questions = df.question.to_list()
# Tokenize sentences
encoded_input = similarity_tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
with torch.no_grad():
    model_output = similarity_model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
sentence_embeddings = sentence_embeddings.detach().numpy()

In [13]:
tokenized_sentences = [text.split(" ") for text in texts]
bm25 = BM25Okapi(tokenized_sentences)

In [14]:
LLM = pipeline(
    model="databricks/dolly-v2-3b", 
    torch_dtype=torch.bfloat16, 
    trust_remote_code=True,
    return_full_text=True,
    device_map="auto",
    task="text-generation",
    temperature=0.9,
)

In [15]:
# template for an instrution with no input
prompt = PromptTemplate(
    input_variables=["instruction"],
    template="{instruction}")

# template for an instruction with input
prompt_with_context = PromptTemplate(
    input_variables=["instruction", "context"],
    template="{instruction}\n\nInput:\n{context}")

hf_pipeline = HuggingFacePipeline(pipeline=LLM)

llm_chain = LLMChain(llm=hf_pipeline, prompt=prompt)
llm_context_chain = LLMChain(llm=hf_pipeline, prompt=prompt_with_context)

In [16]:
query = "Is marble a stron material"
tokenized_query = similarity_tokenizer(query, padding=True, truncation=True, return_tensors='pt')
embedded_query = similarity_model(**tokenized_query)
question_embeddings = mean_pooling(embedded_query, tokenized_query['attention_mask'])
question_embeddings = question_embeddings.detach().numpy()
scores = cosine_similarity([question_embeddings[0]], sentence_embeddings)[0]
max_pos = np.argmax(scores[1:])
max_score = scores[max_pos+1]
similar_answer = texts[max_pos+1]

tokenized_query = query.split(" ")
answer_scores = bm25.get_scores(tokenized_query)
max_score_bm25 = answer_scores.max()
top_answer = bm25.get_top_n(tokenized_query, texts, n=1)[0]

print(f"Query: {query}")
print('------------- Similarity NN -------------')
print(f"Similarity score: {max_score*100:.2f}%")
print(f"Context: {similar_answer}")
print('------------- Similarity BM25 -------------')
print(f"Similarity score: {max_score_bm25}")
print(f"Top answer: {top_answer}")

Query: Is marble a stron material
------------- Similarity NN -------------
Similarity score: 62.08%
Context: the hardness of marble is very high, because the internal structure of the rock is very uniform after longterm natural aging, and the internal stress disappears, so the marble will not be deformed due to temperature, and has strong wear resistance. it is a very popular building material
------------- Similarity BM25 -------------
Similarity score: 8.048500491959693
Top answer: marble is typically not foliated (layered), although there are exceptions.in geology, the term marble refers to metamorphosed limestone, but its use in stonemasonry more broadly encompasses unmetamorphosed limestone. marble is commonly used for sculpture and as a building material


In [18]:
query = "Where there are cracks there is danger?"
tokenized_query = similarity_tokenizer(query, padding=True, truncation=True, return_tensors='pt')
embedded_query = similarity_model(**tokenized_query)
question_embeddings = mean_pooling(embedded_query, tokenized_query['attention_mask'])
question_embeddings = question_embeddings.detach().numpy()
scores = cosine_similarity([question_embeddings[0]], sentence_embeddings)[0]
max_pos = np.argmax(scores[1:])
max_score = scores[max_pos+1]
context = texts[max_pos+1]

print(f'Question: {query}')
print(f'Score: {max_score*100:.2f}')
print(f'Context: {context} \n')
if max_score <= 0.4:
    print("We do not have such context in our knowledge base. Answering with AI without providing it with context, make sure to search the correct answer with critical thinking and research.")
    print(llm_chain.predict(instruction=query).lstrip())
elif max_score <= 0.65 and max_score > 4:
    print("Sorry, i am not exactly sure based on my knowledge base, answering with very low confidence...")
    print(llm_context_chain.predict(instruction=query, context='context').lstrip())
elif max_score <= 0.9 and max_score > 0.65:
    print("Based on the knowledge from database, generating answer...")
    print(llm_context_chain.predict(instruction=query, context=context).lstrip())
else:
    print("Similar question was found with high confidence")
    print(f"Answer: {context}")

Question: Where there are cracks there is danger?
Score: 60.29
Context: most of the cracks have been caused by the rusting and swelling of the iron reinforcements, mainly from the balanos and orlandos restoration . there are also fractures that are the result of mechanical forces or heat (such as earthquakes, the fire in antiquity, and the explosion of 1687) 

Similar question was found with high confidence
Answer: most of the cracks have been caused by the rusting and swelling of the iron reinforcements, mainly from the balanos and orlandos restoration . there are also fractures that are the result of mechanical forces or heat (such as earthquakes, the fire in antiquity, and the explosion of 1687)


In [17]:
query = 'Is kythnos a beautiful island?'
llm_chain.predict(instruction=query).lstrip()

'Yes, kythnos is a beautiful island. The island has many striking features such as black volcanic soil that gives the island a unique look, whitewashed houses, azure seas, lush green vegetation and, of course, beautiful whitewashed houses.'